In [ ]:
import h5py
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import scipy.io

In [ ]:
# read metadata
path = '/yourpath/data/OCT/'

clses = ['AMD', 'Control']
ages = []
paths = []
labels = []
for cls in clses:
    files = os.listdir(path + cls)
    for file in files:
        if '.mat' in file:
            mat = scipy.io.loadmat(os.path.join(path, cls, file))
            img = mat['images']
            age = mat['Age'].item()
            ages.append(age)
            paths.append(os.path.join(path, cls, file))
            label = 0 if cls == 'AMD' else 1
            labels.append(label)
            
            # write to npy
            filename = file.split('.')[0] + '.npy'
            np.save(os.path.join(path, 'images', filename), img)
    


In [ ]:
to_create = list(zip(paths, ages, labels))
meta_all = pd.DataFrame(to_create, columns=['Path', 'Age', 'label'])

In [ ]:
paths = meta_all['Path'].values.tolist()
paths = [x.split('/')[-1].replace('mat', 'npy') for x in paths]
meta_all['Path'] = paths

In [ ]:
meta_all.to_csv('/yourpath/data/OCT/split/meta_all.csv', index = False)

In [ ]:
meta_all['Age_multi'] = meta_all['Age'].values.astype('int')
meta_all['Age_multi'] = np.where(meta_all['Age_multi'].between(0,19), 0, meta_all['Age_multi'])
meta_all['Age_multi'] = np.where(meta_all['Age_multi'].between(20,39), 1, meta_all['Age_multi'])
meta_all['Age_multi'] = np.where(meta_all['Age_multi'].between(40,59), 2, meta_all['Age_multi'])
meta_all['Age_multi'] = np.where(meta_all['Age_multi'].between(60,79), 3, meta_all['Age_multi'])
meta_all['Age_multi'] = np.where(meta_all['Age_multi']>=80, 4, meta_all['Age_multi'])

meta_all['Age_binary'] = meta_all['Age'].values.astype('int')
meta_all['Age_binary'] = np.where(meta_all['Age_binary'].between(0, 70), 0, meta_all['Age_binary'])
meta_all['Age_binary'] = np.where(meta_all['Age_binary']>= 70, 1, meta_all['Age_binary'])
meta_all

In [ ]:
def split_712(all_meta, patient_ids):
    sub_train, sub_val_test = train_test_split(patient_ids, test_size=0.3, random_state=5)
    sub_val, sub_test = train_test_split(sub_val_test, test_size=0.6, random_state=6)
    train_meta = all_meta[all_meta.Path.isin(sub_train)]
    val_meta = all_meta[all_meta.Path.isin(sub_val)]
    test_meta = all_meta[all_meta.Path.isin(sub_test)]
    return train_meta, val_meta, test_meta

sub_train, sub_val, sub_test = split_712(meta_all, np.unique(meta_all['Path']))

In [ ]:
sub_train.to_csv('/yourpath/data/OCT/split/new_train.csv')
sub_val.to_csv('/yourpath/data/OCT/split/new_val.csv')
sub_test.to_csv('/yourpath/data/OCT/split/new_test.csv')